<a href="https://colab.research.google.com/github/ivazin/de_zoomcamp/blob/main/ws1/My_Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [35]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 13
generator = square_root_generator(limit)
s = 0
i = 0

for sqrt_value in generator:
    s += sqrt_value
    i += 1
    print(i, sqrt_value, s)
print("res (2):", s)

1 1.0 1.0
2 1.4142135623730951 2.414213562373095
3 1.7320508075688772 4.146264369941973
4 2.0 6.146264369941973
5 2.23606797749979 8.382332347441762
6 2.449489742783178 10.83182209022494
7 2.6457513110645907 13.47757340128953
8 2.8284271247461903 16.30600052603572
9 3.0 19.30600052603572
10 3.1622776601683795 22.4682781862041
11 3.3166247903554 25.7849029765595
12 3.4641016151377544 29.249004591697254
13 3.605551275463989 32.854555867161245
res (2): 32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [30]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

s = 0
for person in people_1():
    s += person['Age']
    print(person)
print("Ages sum:", s)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    s += person['Age']
    print(person)
print("Ages sum:", s)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Ages sum: 140
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Ages sum: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [33]:
# to do: homework :)

import dlt
import duckdb

table_name = "peoples_data"
pipeline = dlt.pipeline(destination='duckdb', dataset_name='sample_dataset')

# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info1 = pipeline.run(people_1(),
										table_name=table_name,
										write_disposition="merge",
                    primary_key="ID")
print(info1)

info2 = pipeline.run(people_2(),
										table_name=table_name,
										write_disposition="merge",
                    primary_key="ID")
print(info2)

con = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
con.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(con.sql("show tables"))

result = con.execute(f"SELECT * FROM {table_name} ORDER BY id").fetchdf()
result

Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset sample_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707840663.1925752 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset sample_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707840663.945572 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ peoples_data        │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707840663.1925752,X0HzfDzUAtY1Ig,None
1,2,Person_2,27,City_A,1707840663.1925752,UBJ9I2K7oHj0uQ,None
2,3,Person_3,33,City_B,1707840663.945572,JIP5Pc6W+95l8A,Job_3
3,4,Person_4,34,City_B,1707840663.945572,Wf4GgB1Fq2SURQ,Job_4
4,5,Person_5,35,City_B,1707840663.945572,uc4rO9uCRwuWbA,Job_5
5,6,Person_6,36,City_B,1707840663.945572,YY30h9HloCMytw,Job_6
6,7,Person_7,37,City_B,1707840663.945572,QsVFwL6SvKAEsA,Job_7
7,8,Person_8,38,City_B,1707840663.945572,Bjc7cnPyeWAzIw,Job_8


In [34]:
# result = con.execute("SELECT SUM(age) FROM peoples_data").fetchdf()
# result

display(con.sql(f"SELECT SUM(age) FROM {table_name}"))

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX